<a href="https://colab.research.google.com/github/Ameer1503/IoT-Weather-Predictor/blob/main/IoT_Weather_Predictor_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IoT Temperature and Humidity Predictor using AutoTS Algorithm**
AutoTS is an automated time series forecasting library in Python that uses a genetic algorithm to search for the best model parameters for a given dataset. It is built on top of the popular machine learning library scikit-learn and provides a simple, easy-to-use interface for training and evaluating time series models.

AutoTS can handle various time series forecasting tasks, including univariate, multivariate, and forecasting with missing values. It also provides a range of models, including classical statistical models such as ARIMA and exponential smoothing, as well as machine learning models such as Random Forests and XGBoost.

One of the key benefits of AutoTS is its ability to automatically select the best model parameters without requiring manual tuning. This is accomplished by using a genetic algorithm to search for the best model parameters based on a given set of evaluation metrics. This makes it particularly useful for data scientists and analysts who may not have deep expertise in time series modeling but still need to generate accurate forecasts.

Overall, AutoTS is a powerful and flexible tool for time series forecasting that can help users save time and effort in selecting and training models.

In [ ]:
# Installing AutoTS
!pip install autots

In [ ]:
# Declearing Libraries

import pandas as pd
import matplotlib.pyplot as plt
from autots import AutoTS
from autots.datasets import load_daily

In [ ]:
CHANNEL_ID = 2039086

# Load data from ThingSpeak channel as .CSV into dataframe
data = pd.read_csv(f'https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.csv?&results=1000')

In [ ]:
# Convert timestamp to datetime format
data['created_at'] = pd.to_datetime(data['created_at'])

# Set timestamp as index
data.set_index('created_at', inplace=True)

In [ ]:
# Select temperature data and humidity data from dataframe
temp_data = data['field1']
hum_data = data['field2']

## The **AutoTS** object is an instance of the Auto Time Series forecasting library, which is a Python library used for time series forecasting. It is designed to automatically search for the best forecasting model for a given time series dataset by trying different combinations of models and parameters, and evaluating them using various metrics.



### In this specific code, the ** _model object is created using AutoTS with the following parameters:

*   **forecast_length**: the number of time steps to forecast, which is set to 6 to forecast every 4 hours for a day.
*   **frequency**: the frequency of the time series, which is set to 4 hours.
* **prediction_interval**: the prediction interval, which is set to 0.7 to generate a 70% prediction interval for the forecasts.
* **ensemble**: the type of ensemble method to use, which is set to 'simple'
* **model_list**: the list of models to use, which is set to 'multivariate' to use models that can handle multiple input variables.
* **ransformer_list**: the list of transformers to use, which is set to 'superfast' for faster computation.
* **drop_most_recent**: the number of most recent time steps to drop from the input data, which is set to 1.
* **max_generations**: the maximum number of generations for the genetic algorithm used to search for the best models and parameters, which is set to 5.
* **num_validations**: the number of cross-validation folds to use, which is set to 1.
* **models_to_validate**: the proportion of models to validate during the search, which is set to 0.2.
* **n_jobs**: the number of CPU cores to use for parallel computation, which is set to 100.


In [ ]:
# model for temp
temp_model = AutoTS(
    forecast_length=6,  # Forecasting every 4 hours for a day, so 6 forecasts
    frequency='4H',
    prediction_interval=0.7,
    ensemble='simple',
    model_list='multivariate',
    transformer_list='superfast',
    drop_most_recent=1,
    max_generations=5,
    num_validations=1,
    models_to_validate=0.2,
    n_jobs=100,
)


# model for humidity
hum_model = AutoTS(
    forecast_length=6,  # Forecasting every 4 hours for a day, so 6 forecasts
    frequency='4H',
    prediction_interval=0.7,
    ensemble='simple',
    model_list='multivariate',
    transformer_list='superfast',
    drop_most_recent=1,
    max_generations=5,
    num_validations=1,
    models_to_validate=0.2,
    n_jobs=100,
)


#**Temprature forecast**

###**temp_model.fit(temp_data)**
is a method call to fit the **temp_data** into the **temp_model** instance using the AutoTS algorithm. This method trains the model and finds the best-fitted model based on the given input data and parameters.

###**temp_forecast = temp_model.predict()**
forecast is a method call to make a forecast using the **temp_model** instance. This method uses the trained model to make future predictions and returns the forecasted values for the specified **forecast_length**. In this case, the forecast length is set to 6, which means the method returns 6 forecasted values for temperature at an interval of every 4 hours.

In [ ]:
temp_model.fit(temp_data)
temp_forecast = temp_model.predict().forecast

temp_model_results = temp_model.results()

In [ ]:
#Validation the the Temprature Results
temp_model.results("validation")

#**Humidity forecast**
###**hum_model.fit(hum_data)**
is a method call to fit the **hum_data** into the **hum_model** instance using the AutoTS algorithm. This method trains the model and finds the best-fitted model based on the given input data and parameters.

###**hum_forecast = hum_model.predict()**
forecast is a method call to make a forecast using the **hum_model** instance. This method uses the trained model to make future predictions and returns the forecasted values for the specified **forecast_length**. In this case, the forecast length is set to 6, which means the method returns 6 forecasted values for temperature at an interval of every 4 hours.

In [ ]:
hum_model.fit(hum_data)
hum_forecast = hum_model.predict().forecast

hum_model_results = hum_model.results()

In [ ]:
#Validation the the Humidity Results
hum_model.results("validation")

#**Ploting the Forcasted Temprature and Humidity**

In [ ]:
# Plot actual and forecasted temperature and humidity
fig, ax = plt.subplots(figsize=(20, 6))
ax.plot(temp_data.index, temp_data, label='Temperature Actual')
ax.plot(temp_forecast.index, temp_forecast, label='Temperature Forecast')
ax.plot(hum_data.index, hum_data, label='Humidity Actual')
ax.plot(hum_forecast.index, hum_forecast, label='Humidity Forecast')

#**Plot upper and lower forecast bounds as shaded region**

In [ ]:
# Plot upper and lower forecast bounds as shaded region
fig, ax = plt.subplots(figsize=(20, 6))
temp_up, temp_low = temp_model.predict().upper_forecast.squeeze(), temp_model.predict().lower_forecast.squeeze()
hum_up, hum_low = hum_model.predict().upper_forecast.squeeze(), hum_model.predict().lower_forecast.squeeze()
ax.fill_between(temp_up.index, temp_up.values, temp_low.values, alpha=0.6)
ax.fill_between(hum_up.index, hum_up.values, hum_low.values, alpha=0.6)

ax.legend()
plt.show()

##**The Date and Time format is**

###**03-24 20**
###**MM-DD HH**


---

###**03**: MONTH
###**24**: DAY
###**20**: HOUR

